In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from datetime import datetime, timedelta

%reload_ext dotenv

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [11]:
# login
url = '''https://www.linkedin.com/company/soujunior/posts/?feedView=all'''
driver.get(url)

try:
    xpath_sign = """//*[@id="main-content"]/div[1]/form/p/button"""
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_sign))).click()
except:
    pass

xpath_login = """//*[@id="session_key"]"""
xpath_password = """//*[@id="session_password"]"""
xpath_submit = """//*[@id="main-content"]/div[1]/div/form/div[2]/button"""

login_linkedin = os.getenv("LOGIN_LINKEDIN")
password_linkedin = os.getenv("PASSWORD_LINKEDIN")

input_login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_login)))
input_password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_password)))
input_submit = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_submit)))

input_login.clear()
input_password.clear()

input_login.send_keys(login_linkedin)
input_password.send_keys(password_linkedin)
input_submit.click()

time.sleep(3)
driver.get(url)

KeyboardInterrupt: 

In [17]:
# sort by date
class_sort = "sort-dropdown__dropdown"

sort_div = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, class_sort))
)

sort_button = sort_div.find_element(By.TAG_NAME, "button")
sort_button.click()

sort_by_date = sort_div.find_elements(By.TAG_NAME, "li")[1]
sort_by_date.click()

time.sleep(2)

# scroll down to load more posts
body = driver.find_element(By.TAG_NAME, "body")
for _ in range(2):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [20]:
class_feed_posts = "scaffold-finite-scroll__content"

feed_posts = driver.find_element(By.CLASS_NAME, class_feed_posts)

posts = feed_posts.find_elements(By.XPATH, './div')

In [21]:
#save posts to html
output_folder = "new_contents"

screenshoted_author_list = []
for index, post in enumerate(posts):

    author = post.find_elements(By.CLASS_NAME, "update-components-actor__name")

    # validation
    if author:
        author_name = author[0].find_element(By.CLASS_NAME, "visually-hidden").text
    else:
        continue
    
    if author_name not in screenshoted_author_list:
        img = post.find_element(By.CLASS_NAME, 'update-components-actor__avatar-image')
        img_path = f"linkedin_page/scraps/{output_folder}/profile_pics/{author_name}.png"
        img.screenshot(f"linkedin_page/scraps/{output_folder}/profile_pics/{author_name}.png")

        screenshoted_author_list.append(author_name)

    # save post
    with open(f'linkedin_page/scraps/{output_folder}/post_{index}.html', 'w', encoding='utf-8') as file:
        file.write(f'<p>{author_name}</p>\n')
        file.write(post.get_attribute('outerHTML'))

### Veryfy and append new posts

In [24]:
posts_html_path = "linkedin_page/scraps/new_contents/"

html_files_path = [file for file in os.listdir(posts_html_path) if file.endswith(".html")]
soup_list = []
for html in html_files_path:
    filename = html
    with open(posts_html_path + filename, "r", encoding="utf8") as file:
        soup_list.append(BeautifulSoup(file, "html.parser"))

In [33]:
main_df = pd.read_csv('linkedin_page/dataframes/main_dataframe.csv')
main_df_post_ids = main_df['linkedin_id'].tolist()

In [28]:
today_date = datetime.now()

def define_date(raw_date):
    num, mult, _ = raw_date.split(" ")
    if mult == "d":
        publish_date = today_date - timedelta(days=int(num))
    elif mult == "sem":
        publish_date = today_date - timedelta(days=int(num) * 7)
    elif mult == "m":
        publish_date = today_date - timedelta(days=int(num) * 30)
    elif mult == "h":
        publish_date = today_date

    return publish_date.date()

In [47]:
df_dict = {
    "publish_date": [],
    "shared_post": [],
    "author_name": [],
    "author_profile_url": [],
    "post_type": [],
    "post_url": [],
    "post_content": [],
    "reaction_count": [],
    "comment_count": [],
    "share_count": [],
    'linkedin_id': [],
}
corr_index_filename = []


for index, soup in enumerate(soup_list):
    # print(soup.prettify())
    publish_date = (
        soup.find_all(class_="update-components-actor__meta")[0]
        .find_all(class_="visually-hidden")[-1]
        .text.split("•")[0]
        if soup.find_all(class_="update-components-actor__meta") != []
        else None
        if (soup.find(class_="update-components-promo-v1__text-container") != None)
        or (len(soup.find_all(tag="div")) <= 7)
        else "Unknown"
    )

    # Linkedin Promo or Unloaded scraps
    if publish_date == None:
        continue

    publish_date = define_date(publish_date)

    shared_post = (
        True
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else False
    )

    author_name = soup.find(class_="update-components-actor__name").span.span.text

    author_profile_url = soup.find(class_="update-components-actor__meta-link")["href"]

    post_type = (
        "Shared Content"
        if (soup.find(class_="update-components-header__text-view") is not None)
        or (
            "feed-shared-update-v2__update-content-wrapper"
            in soup.find(
                class_="feed-shared-update-v2__description-wrapper"
            ).find_next_sibling()["class"]
        )
        else "Poll"
        if soup.find(class_="update-components-poll") is not None
        else "Scheduled Event"
        if soup.find(class_="update-components-scheduled-live-content__event-link")
        is not None
        else "Live"
        if soup.find(class_="video-live") is not None
        else "Single Image"
        if soup.find(class_="update-components-image--single-image") is not None
        else "Text only"
        if "update-v2-social-activity"
        in soup.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
        else "Default"
    )

    post_url = (
        "https://www.linkedin.com/feed/update/"
        + soup.find(class_="feed-shared-update-v2")["data-urn"]
    )

    post_content = (
        soup.find(class_="update-components-update-v2__commentary").span.span.text
        if post_type == "Shared Content"
        else soup.find(class_="update-components-text").span.span.text
    )

    # Reaction
    if soup.find(class_="social-details-social-counts__reactions-count") != None:
        reaction_count = soup.find(
            class_="social-details-social-counts__reactions-count"
        ).text.strip()

    else:
        qtd_reactions = soup.find(
            class_="social-details-social-counts__social-proof-text"
        ).text.strip()

        re_pattern_reactions = r"\b(\d+)\s+pessoas\b"

        reaction_count = 1 + int(re.findall(re_pattern_reactions, qtd_reactions)[0])

    comment_count = (
        soup.find(
            class_="social-details-social-counts__comments"
        ).button.span.text.strip().split(' ')[0]
        if soup.find(class_="social-details-social-counts__comments") != None
        else "0"
    )

    share_count = (
        0
        if len(
            soup.find_all(class_="social-details-social-counts__item--right-aligned")
        )
        <= 1
        else soup.find_all(class_="social-details-social-counts__item--right-aligned")[
            1
        ]
        .button.span.text.strip()
        .split(" ")[0]
    )

    linkedin_id = post_url.split(':')[-1]

    if int(linkedin_id) in main_df_post_ids:
        continue

    df_dict["publish_date"].append(publish_date)
    df_dict["shared_post"].append(shared_post)
    df_dict["author_name"].append(author_name)
    df_dict["author_profile_url"].append(author_profile_url)
    df_dict["post_type"].append(post_type)
    df_dict["post_url"].append(post_url)
    df_dict["post_content"].append(post_content)
    df_dict["reaction_count"].append(reaction_count)
    df_dict["comment_count"].append(comment_count)
    df_dict["share_count"].append(share_count)
    df_dict['linkedin_id'].append(linkedin_id)

df = pd.DataFrame(df_dict)

já tem
já tem
já tem
já tem
já tem
já tem
já tem
já tem


In [48]:
#----------------DEBUG----------------

df

,publish_date,shared_post,author_name,author_profile_url,post_type,post_url,post_content,reaction_count,comment_count,share_count,linkedin_id
0,2023-12-11,True,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,🌟 Anúncio de Evento Especial: Desvendando o Mu...,3,0,0,7140031369270779904
